In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import sys, os
import numpy as np
import matplotlib.pyplot as plt

In [23]:
# parallel processing
from multiprocessing import Pool
Nprocs = 4

In [4]:
# samwich routines
samwich_path = os.path.join(os.environ['HOME'],'waketracking')
if not samwich_path in sys.path:
    sys.path.append(samwich_path)
from samwich.dataloaders import XarrayData
from samwich.waketrackers import track

In [5]:
# case definition
from study import V27, neutral

# Wake tracking with all methods

## set up tracking parameters

In [6]:
case = neutral(casedir='WAKEBENCH-NREL_DEBNATH_LES/neutral',
               prefix='NREL_DEBNATH_LES_20190325',
               turbine=V27)

In [7]:
Aref = case.turbine.rotor_area

In [8]:
from samwich.gaussian_functions import Bastankhah
gauss = Bastankhah(CT=case.turbine.CTref, d0=case.turbine.D, TI=case.TI/100)

Calculated k* : 0.0447339


## read all planes

In [10]:
%%time
dataplanes = {
    downD: XarrayData(case.get_wake_datafile(downD),
                      uvar='U',vvar='V',wvar='W',
                      trim_time=case.trim_time(downD))
    for downD in case.downstreamD
}

Selected datafile: WAKEBENCH-NREL_DEBNATH_LES/neutral/NREL_DEBNATH_LES_20190325_uvw_2D.nc
Calculated offset: 13
Selected datafile: WAKEBENCH-NREL_DEBNATH_LES/neutral/NREL_DEBNATH_LES_20190325_uvw_3D.nc
Calculated offset: 17
Selected datafile: WAKEBENCH-NREL_DEBNATH_LES/neutral/NREL_DEBNATH_LES_20190325_uvw_4D.nc
Calculated offset: 20
Selected datafile: WAKEBENCH-NREL_DEBNATH_LES/neutral/NREL_DEBNATH_LES_20190325_uvw_5D.nc
Calculated offset: 23
Selected datafile: WAKEBENCH-NREL_DEBNATH_LES/neutral/NREL_DEBNATH_LES_20190325_uvw_6D.nc
Calculated offset: 26
Selected datafile: WAKEBENCH-NREL_DEBNATH_LES/neutral/NREL_DEBNATH_LES_20190325_uvw_7D.nc
Calculated offset: 29
Selected datafile: WAKEBENCH-NREL_DEBNATH_LES/neutral/NREL_DEBNATH_LES_20190325_uvw_8D.nc
Calculated offset: 32
CPU times: user 7.3 s, sys: 6.83 s, total: 14.1 s
Wall time: 14.2 s


## run trackers

In [11]:
def track_downstream_plane(downD,**tracking_params):
    name = tracking_params.pop('name')
    method = tracking_params.pop('method')
    data = dataplanes[downD]
    # evaluate parameters that are a function of downstream distance
    for key,val in tracking_params.items():
        if isinstance(val, dict):
            tracking_params[key] = val[downD]
    print('x/D:',downD,':',tracking_params)
    # perform the wake tracking
    wo = track(data.sliceI(), method=method)
    wo.remove_shear(wind_profile=case.get_inflow(downD))
    yc,zc = wo.find_centers(**tracking_params, **case.get_outputs(name,downD))

### Gaussian 2D

In [12]:
Gaussian2D_params = {
    'name': '2D Gaussian',
    'method': 'Gaussian2D',
    'umin': None, # use VD minima in each snapshot
    'A_ref': Aref,
    'A_min': Aref/5,
    'A_max': 2*Aref,
    'AR_max': 10.0,
}

In [13]:
p = Pool(Nprocs)
%time p.map(partial(track_downstream_plane, **Gaussian2D_params), case.downstreamD)
# Nprocs=4
# CPU times: user 490 ms, sys: 203 ms, total: 693 ms
# Wall time: 45min 36s

x/D: 2 : {'umin': None, 'A_ref': 572.5552611167398, 'A_min': 114.51105222334795, 'A_max': 1145.1105222334795, 'AR_max': 10.0}
x/D: 3 : {'umin': None, 'A_ref': 572.5552611167398, 'A_min': 114.51105222334795, 'A_max': 1145.1105222334795, 'AR_max': 10.0}
x/D: 5 : {'umin': None, 'A_ref': 572.5552611167398, 'A_min': 114.51105222334795, 'A_max': 1145.1105222334795, 'AR_max': 10.0}
x/D: 4 : {'umin': None, 'A_ref': 572.5552611167398, 'A_min': 114.51105222334795, 'A_max': 1145.1105222334795, 'AR_max': 10.0}
Slicing data at i=0 x=54.0
Slicing data at i=0 x=81.0
Slicing data at i=0 x=108.0
Slicing data at i=0 x=135.0
Selected Tracker: Gaussian2D
Selected Tracker: Gaussian2D
Selected Tracker: Gaussian2D



Sampling plane normal vector: [1. 0. 0.]
Sampling plane normal vector: [1. 0. 0.]
Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 135.0 0.0 40.5010165
  identified plane center at: 108.0 0.0 40.5010165
  identified plane center at: 81.0 0.0 40.5010165
Selected Tracker: Gau

[None, None, None, None, None, None, None]

Process ForkPoolWorker-2:
Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Traceback (most recent call last):
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
Traceback (most recent call last):
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/equon/miniconda3/lib/python3.7/multiprocessin

### Gaussian 1D

In [14]:
Bastankhah_params = {
    'name': '1D Gaussian (Bastankhah)',
    'method': 'Gaussian',
    'umin': { x_D: gauss.amplitude(x_D*case.turbine.D,-case.Uref) for x_D in case.downstreamD },
    'sigma': { x_D: gauss.sigma(x_D*case.turbine.D) for x_D in case.downstreamD },
}

Calculated Gaussian width : 8.598824208554174 m
Calculated Gaussian amplitude : -4.474464240770766 m/s
Calculated Gaussian width : 9.806639508554174 m
Calculated Gaussian amplitude : -3.112149184268989 m/s
Calculated Gaussian width : 11.014454808554174 m
Calculated Gaussian amplitude : -2.340643747870897 m/s
Calculated Gaussian width : 12.222270108554174 m
Calculated Gaussian amplitude : -1.8396952310704735 m/s
Calculated Gaussian width : 13.430085408554174 m
Calculated Gaussian amplitude : -1.4902034362444538 m/s
Calculated Gaussian width : 14.637900708554174 m
Calculated Gaussian amplitude : -1.2345920413753313 m/s
Calculated Gaussian width : 15.845716008554174 m
Calculated Gaussian amplitude : -1.0410934863247117 m/s
Calculated Gaussian width : 8.598824208554174 m
Calculated Gaussian width : 9.806639508554174 m
Calculated Gaussian width : 11.014454808554174 m
Calculated Gaussian width : 12.222270108554174 m
Calculated Gaussian width : 13.430085408554174 m
Calculated Gaussian width :

In [15]:
p = Pool(Nprocs)
%time p.map(partial(track_downstream_plane, **Bastankhah_params), case.downstreamD)
# Nprocs=4
# CPU times: user 150 ms, sys: 79.8 ms, total: 230 ms
# Wall time: 7min 36s

x/D: 3 : {'umin': -3.112149184268989, 'sigma': 9.806639508554174}
x/D: 2 : {'umin': -4.474464240770766, 'sigma': 8.598824208554174}
x/D: 4 : {'umin': -2.340643747870897, 'sigma': 11.014454808554174}
x/D: 5 : {'umin': -1.8396952310704735, 'sigma': 12.222270108554174}
Slicing data at i=0 x=108.0
Slicing data at i=0 x=135.0
Slicing data at i=0 x=81.0
Slicing data at i=0 x=54.0
Selected Tracker: Gaussian
Selected Tracker: Gaussian
Selected Tracker: Gaussian



Sampling plane normal vector: [1. 0. 0.]
Sampling plane normal vector: [1. 0. 0.]
Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 108.0 0.0 40.5010165
  identified plane center at: 81.0 0.0 40.5010165
  identified plane center at: 135.0 0.0 40.5010165
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotation error: 0.0
  rotation error: 0.0
  rotation error: 0.0
  horizontal search range: -40.0 40.0
  horiz

[None, None, None, None, None, None, None]

Process ForkPoolWorker-8:
Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Process ForkPoolWorker-7:
Traceback (most recent call last):
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(

### Constant area

In [16]:
const_area_params = {
    'name': 'Constant-Area Contours',
    'method': 'ConstantArea',
    'ref_area': Aref,
}

In [17]:
p = Pool(Nprocs)
%time p.map(partial(track_downstream_plane, **const_area_params), case.downstreamD)
# Nprocs=4
# CPU times: user 136 ms, sys: 32.7 ms, total: 169 ms
# Wall time: 4min 1s

x/D: 2 : {'ref_area': 572.5552611167398}
x/D: 3 : {'ref_area': 572.5552611167398}
x/D: 4 : {'ref_area': 572.5552611167398}
x/D: 5 : {'ref_area': 572.5552611167398}
Slicing data at i=0 x=54.0
Slicing data at i=0 x=81.0
Slicing data at i=0 x=108.0
Slicing data at i=0 x=135.0
Selected Tracker: ConstantArea
Selected Tracker: ConstantArea
Selected Tracker: ConstantArea
Selected Tracker: ConstantArea




Sampling plane normal vector: [1. 0. 0.]
Sampling plane normal vector: [1. 0. 0.]
Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 81.0 0.0 40.5010165
  identified plane center at: 54.0 0.0 40.5010165
  identified plane center at: 135.0 0.0 40.5010165
Sampling plane normal vector: [1. 0. 0.]
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotated to rotor-aligned axes (about z): 0.0 deg
  identified plane center at: 108.0 0.0 40.5010165
  rotation error: 0.0
  rotation error: 0.0
  rotation error: 0.0
  horizontal 

[None, None, None, None, None, None, None]

Process ForkPoolWorker-9:
Process ForkPoolWorker-10:
Process ForkPoolWorker-12:
Process ForkPoolWorker-11:
Traceback (most recent call last):
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/equon/miniconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = re

### Constant flux
Recall: definition of velocity fields in SAMWICH Box is $u_{tot} = u_{avg} + u$

momentum flux $M = U_w(U_\infty - U_w) = -(U_w - U_\infty)U_w = $ `-u*u_tot`, for wake velocity $U_w$

In [36]:
def fluxfun(u,u_w):
    """x-momentum flux
    Note: function arguments correspond to `field_names` kwarg
    """
    return -u * u_w 

In [37]:
const_xmom_params = {
    'name': 'Constant-Xmom Contours',
    'method': 'ConstantFlux',
    'ref_flux': case.turbine.thrust(case.Uref),
    'flux_function': fluxfun,
    'field_names': ('u','u_tot'),
}

In [38]:
p = Pool(Nprocs)
%time p.map(partial(track_downstream_plane, **const_xmom_params), case.downstreamD)
# Nprocs=4
# CPU times: user 35.5 s, sys: 18.3 s, total: 53.9 s
# Wall time: 6h 13min 54s

x/D: 2 : {'ref_flux': 13434.379391317067, 'flux_function': <function fluxfun at 0x7f0fb8fc1d08>, 'field_names': ('u', 'u_tot')}
x/D: 4 : {'ref_flux': 13434.379391317067, 'flux_function': <function fluxfun at 0x7f0fb8fc1d08>, 'field_names': ('u', 'u_tot')}
x/D: 3 : {'ref_flux': 13434.379391317067, 'flux_function': <function fluxfun at 0x7f0fb8fc1d08>, 'field_names': ('u', 'u_tot')}
x/D: 5 : {'ref_flux': 13434.379391317067, 'flux_function': <function fluxfun at 0x7f0fb8fc1d08>, 'field_names': ('u', 'u_tot')}
Slicing data at i=0 x=54.0
Slicing data at i=0 x=108.0
Slicing data at i=0 x=135.0
Slicing data at i=0 x=81.0
Selected Tracker: ConstantFlux
Selected Tracker: ConstantFlux
Selected Tracker: ConstantFlux
Selected Tracker: ConstantFlux




Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 81.0 0.0 40.5010165
Sampling plane normal vector: [1. 0. 0.]
Sampling plane normal vector: [1. 0. 0.]
Sampling plane normal vector: [1. 0. 0.]
  rotated to rotor-aligned axes (abo

Processed frame 1590


Wrote out trajectory to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Constant-Xmom_Contours/trajectory_2D.csv
Wrote out pickled outlines to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Constant-Xmom_Contours/outlines_2D.pkl
x/D: 6 : {'ref_flux': 13434.379391317067, 'flux_function': <function fluxfun at 0x7f0fb8fc1d08>, 'field_names': ('u', 'u_tot')}
Slicing data at i=0 x=162.0
Selected Tracker: ConstantFlux

Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 162.0 0.0 40.5010165
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotation error: 0.0
  horizontal search range: -40.0 40.0
  vertical search range: 0.497933 80.5041
Number of time frames to process: 1578

...finished initializing WakeTracker

...finished initializing ContourWakeTracker

...finished initializing ConstantFlux 

  subtracting out time-varying profile
Sample function evaluation: f(u=-3.3272677798353643,u_tot=2.34204) = 7.792594231085616
Attempting to match flux: 13434.379391317067


Processed frame 1586


Wrote out trajectory to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Constant-Xmom_Contours/trajectory_3D.csv
Wrote out pickled outlines to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Constant-Xmom_Contours/outlines_3D.pkl
x/D: 7 : {'ref_flux': 13434.379391317067, 'flux_function': <function fluxfun at 0x7f0fb8fc1d08>, 'field_names': ('u', 'u_tot')}
Slicing data at i=0 x=189.0
Selected Tracker: ConstantFlux

Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 189.0 0.0 40.5010165
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotation error: 0.0
  horizontal search range: -40.0 40.0
  vertical search range: 0.497933 80.5041
Number of time frames to process: 1575

...finished initializing WakeTracker

...finished initializing ContourWakeTracker

...finished initializing ConstantFlux 

  subtracting out time-varying profile
Sample function evaluation: f(u=-2.9099898179012564,u_tot=2.08281) = 6.060955892622915
Attempting to match flux: 13434.379391317067


...

Processed frame 1574


Wrote out trajectory to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Constant-Xmom_Contours/trajectory_7D.csv
Wrote out pickled outlines to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Constant-Xmom_Contours/outlines_7D.pkl


Processed frame 1571


Wrote out trajectory to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Constant-Xmom_Contours/trajectory_8D.csv
Wrote out pickled outlines to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Constant-Xmom_Contours/outlines_8D.pkl
CPU times: user 35.5 s, sys: 18.3 s, total: 53.9 s
Wall time: 6h 13min 54s


[None, None, None, None, None, None, None]